# Coursera project week3, Problem 3
In this file, I explore and cluster the neighborhoods in Etobicoke.


In [1]:
import pandas as pd
import numpy as np
import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

from geopy.geocoders import Nominatim

# import k-means from clustering stage
from sklearn.cluster import KMeans
import folium # map rendering library

VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

df_new = pd.read_csv('data.csv') # Postal codes and corresponding geospatial coordinates of Canada
print(df_new)

    Postal_Code           Borough  \
0           M3A        North York   
1           M4A        North York   
2           M5A  Downtown Toronto   
3           M6A        North York   
4           M7A  Downtown Toronto   
..          ...               ...   
98          M8X         Etobicoke   
99          M4Y  Downtown Toronto   
100         M7Y      East Toronto   
101         M8Y         Etobicoke   
102         M8Z         Etobicoke   

                                         Neighbourhood   Latitude  Longitude  
0                                            Parkwoods  43.753259 -79.329656  
1                                     Victoria Village  43.725882 -79.315572  
2                            Regent Park, Harbourfront  43.654260 -79.360636  
3                     Lawrence Manor, Lawrence Heights  43.718518 -79.464763  
4          Queen's Park, Ontario Provincial Government  43.662301 -79.389494  
..                                                 ...        ...        ...  
98

In [2]:
# function
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            'HM3AAQI50VK0VMIFVRGE1WATLARDR3EHLEYY1RAVJFN3PRKE', 
            'ENWI1Y4UVIVQ13MEKRL5FKYYTHNE5JAXRF2JASLIOG05JRX4', 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [3]:
address = 'ETOBICOKE, Ontario Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Etobicoke are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Etobicoke are 43.6435559, -79.5656326.


In [8]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)
# create map of Etobicoke using latitude and longitude values
map_Etobicoke = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Etobicoke)  
    
map_Etobicoke

In [9]:
Etobicoke_data = df_new[df_new['Borough'] == 'Etobicoke'].reset_index(drop=True) #12 rows
Etobicoke_data = Etobicoke_data.rename(columns = {"Neighbourhood":"Neighborhood"}) 
Etobicoke_data.head()

,Postal_Code,Borough,Neighborhood,Latitude,Longitude
0,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
1,M9B,Etobicoke,"West Deane Park, Princess Gardens, Martin Grov...",43.650943,-79.554724
2,M9C,Etobicoke,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",43.643515,-79.577201
3,M9P,Etobicoke,Westmount,43.696319,-79.532242
4,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724


In [10]:
Etobicoke_venues = getNearbyVenues(names=Etobicoke_data['Neighborhood'],
                                   latitudes=Etobicoke_data['Latitude'],
                                   longitudes=Etobicoke_data['Longitude']
                                  )

Islington Avenue, Humber Valley Village
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Westmount
Kingsview Village, St. Phillips, Martin Grove Gardens, Richview Gardens
New Toronto, Mimico South, Humber Bay Shores
South Steeles, Silverstone, Humbergate, Jamestown, Mount Olive, Beaumond Heights, Thistletown, Albion Gardens
Alderwood, Long Branch
Northwest, West Humber - Clairville
The Kingsway, Montgomery Road, Old Mill North
Old Mill South, King's Mill Park, Sunnylea, Humber Bay, Mimico NE, The Queensway East, Royal York South East, Kingsway Park South East
Mimico NW, The Queensway West, South of Bloor, Kingsway Park South West, Royal York South West


In [11]:
Etobicoke_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Alderwood, Long Branch",8,8,8,8,8,8
"Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood",8,8,8,8,8,8
"Kingsview Village, St. Phillips, Martin Grove Gardens, Richview Gardens",4,4,4,4,4,4
"Mimico NW, The Queensway West, South of Bloor, Kingsway Park South West, Royal York South West",14,14,14,14,14,14
"New Toronto, Mimico South, Humber Bay Shores",11,11,11,11,11,11
"Northwest, West Humber - Clairville",4,4,4,4,4,4
"Old Mill South, King's Mill Park, Sunnylea, Humber Bay, Mimico NE, The Queensway East, Royal York South East, Kingsway Park South East",2,2,2,2,2,2
"South Steeles, Silverstone, Humbergate, Jamestown, Mount Olive, Beaumond Heights, Thistletown, Albion Gardens",10,10,10,10,10,10
"The Kingsway, Montgomery Road, Old Mill North",1,1,1,1,1,1


In [12]:
print('There are {} uniques categories.'.format(len(Etobicoke_venues['Venue Category'].unique())))

There are 40 uniques categories.


In [13]:
# one hot encoding
Etobicoke_onehot = pd.get_dummies(Etobicoke_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Etobicoke_onehot['Neighborhood'] = Etobicoke_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Etobicoke_onehot.columns[-1]] + list(Etobicoke_onehot.columns[:-1])
Etobicoke_onehot = Etobicoke_onehot[fixed_columns]

Etobicoke_onehot.head()

,Neighborhood,American Restaurant,Bakery,Baseball Field,Beer Store,Brewery,Burger Joint,Bus Line,Café,Chinese Restaurant,...,Rental Car Location,Restaurant,River,Sandwich Place,Skating Rink,Supplement Shop,Tanning Salon,Thrift / Vintage Store,Truck Stop,Wings Joint
0,"West Deane Park, Princess Gardens, Martin Grov...",0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"West Deane Park, Princess Gardens, Martin Grov...",0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
Etobicoke_onehot.shape

(71, 41)

In [15]:
Etobicoke_grouped = Etobicoke_onehot.groupby('Neighborhood').mean().reset_index()
Etobicoke_grouped

,Neighborhood,American Restaurant,Bakery,Baseball Field,Beer Store,Brewery,Burger Joint,Bus Line,Café,Chinese Restaurant,...,Rental Car Location,Restaurant,River,Sandwich Place,Skating Rink,Supplement Shop,Tanning Salon,Thrift / Vintage Store,Truck Stop,Wings Joint
0,"Alderwood, Long Branch",0.000000,0.000000,0.0,0.000,0.0,0.000000,0.00,0.000000,0.000000,...,0.00,0.000000,0.0,0.125000,0.125,0.000000,0.000000,0.000000,0.00,0.000000
1,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",0.000000,0.000000,0.0,0.125,0.0,0.000000,0.00,0.125000,0.000000,...,0.00,0.000000,0.0,0.000000,0.000,0.000000,0.000000,0.000000,0.00,0.000000
2,"Kingsview Village, St. Phillips, Martin Grove ...",0.000000,0.000000,0.0,0.000,0.0,0.000000,0.25,0.000000,0.000000,...,0.00,0.000000,0.0,0.250000,0.000,0.000000,0.000000,0.000000,0.00,0.000000
3,"Mimico NW, The Queensway West, South of Bloor,...",0.000000,0.071429,0.0,0.000,0.0,0.071429,0.00,0.000000,0.000000,...,0.00,0.000000,0.0,0.071429,0.000,0.071429,0.071429,0.071429,0.00,0.071429
4,"New Toronto, Mimico South, Humber Bay Shores",0.090909,0.090909,0.0,0.000,0.0,0.000000,0.00,0.090909,0.000000,...,0.00,0.090909,0.0,0.000000,0.000,0.000000,0.000000,0.000000,0.00,0.000000
5,"Northwest, West Humber - Clairville",0.000000,0.000000,0.0,0.000,0.0,0.000000,0.00,0.000000,0.000000,...,0.25,0.000000,0.0,0.000000,0.000,0.000000,0.000000,0.000000,0.25,0.000000
6,"Old Mill South, King's Mill Park, Sunnylea, Hu...",0.000000,0.000000,0.5,0.000,0.0,0.000000,0.00,0.000000,0.000000,...,0.00,0.000000,0.0,0.000000,0.000,0.000000,0.000000,0.000000,0.00,0.000000
7,"South Steeles, Silverstone, Humbergate, Jamest...",0.000000,0.000000,0.0,0.100,0.0,0.000000,0.00,0.000000,0.000000,...,0.00,0.000000,0.0,0.100000,0.000,0.000000,0.000000,0.000000,0.00,0.000000
8,"The Kingsway, Montgomery Road, Old Mill North",0.000000,0.000000,0.0,0.000,0.0,0.000000,0.00,0.000000,0.000000,...,0.00,0.000000,1.0,0.000000,0.000,0.000000,0.000000,0.000000,0.00,0.000000
9,"West Deane Park, Princess Gardens, Martin Grov...",0.000000,0.500000,0.0,0.000,0.5,0.000000,0.00,0.000000,0.000000,...,0.00,0.000000,0.0,0.000000,0.000,0.000000,0.000000,0.000000,0.00,0.000000


In [16]:
Etobicoke_grouped.shape

(11, 41)

In [17]:
num_top_venues = 5

for hood in Etobicoke_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Etobicoke_grouped[Etobicoke_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Alderwood, Long Branch----
            venue  freq
0     Pizza Place  0.25
1             Pub  0.12
2             Gym  0.12
3    Skating Rink  0.12
4  Sandwich Place  0.12


----Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood----
          venue  freq
0  Liquor Store  0.12
1    Beer Store  0.12
2          Park  0.12
3     Pet Store  0.12
4          Café  0.12


----Kingsview Village, St. Phillips, Martin Grove Gardens, Richview Gardens----
                 venue  freq
0    Mobile Phone Shop  0.25
1             Bus Line  0.25
2       Sandwich Place  0.25
3          Pizza Place  0.25
4  American Restaurant  0.00


----Mimico NW, The Queensway West, South of Bloor, Kingsway Park South West, Royal York South West----
                    venue  freq
0              Kids Store  0.07
1          Discount Store  0.07
2  Thrift / Vintage Store  0.07
3           Tanning Salon  0.07
4         Supplement Shop  0.07


----New Toronto, Mimico South, Humber Bay Shores----
             

In [18]:
#a function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [19]:
num_top_venues = 10 # top 10 venues for each neighborhood

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Etobicoke_grouped['Neighborhood']

for ind in np.arange(Etobicoke_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Etobicoke_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Alderwood, Long Branch",Pizza Place,Sandwich Place,Coffee Shop,Pharmacy,Pub,Gym,Skating Rink,Bakery,Garden Center,Fried Chicken Joint
1,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",Pizza Place,Pet Store,Beer Store,Liquor Store,Café,Coffee Shop,Park,Pharmacy,Fried Chicken Joint,Fast Food Restaurant
2,"Kingsview Village, St. Phillips, Martin Grove ...",Sandwich Place,Mobile Phone Shop,Bus Line,Pizza Place,Wings Joint,Coffee Shop,Garden Center,Fried Chicken Joint,Fast Food Restaurant,Drugstore
3,"Mimico NW, The Queensway West, South of Bloor,...",Wings Joint,Burger Joint,Gym,Hardware Store,Kids Store,Fast Food Restaurant,Discount Store,Convenience Store,Grocery Store,Sandwich Place
4,"New Toronto, Mimico South, Humber Bay Shores",American Restaurant,Restaurant,Fast Food Restaurant,Liquor Store,Mexican Restaurant,Pet Store,Pharmacy,Café,Pizza Place,Bakery


In [20]:
# set number of clusters
kclusters = 5

Etobicoke_grouped_clustering = Etobicoke_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Etobicoke_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 2, 2, 0, 3, 2, 4, 1], dtype=int32)

In [21]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Etobicoke_merged = Etobicoke_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
Etobicoke_merged = Etobicoke_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Etobicoke_merged.head() # check the last columns!

,Postal_Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,M9B,Etobicoke,"West Deane Park, Princess Gardens, Martin Grov...",43.650943,-79.554724,1.0,Bakery,Brewery,Wings Joint,Convenience Store,Gym,Grocery Store,Garden Center,Fried Chicken Joint,Fast Food Restaurant,Drugstore
2,M9C,Etobicoke,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",43.643515,-79.577201,2.0,Pizza Place,Pet Store,Beer Store,Liquor Store,Café,Coffee Shop,Park,Pharmacy,Fried Chicken Joint,Fast Food Restaurant
3,M9P,Etobicoke,Westmount,43.696319,-79.532242,2.0,Intersection,Sandwich Place,Middle Eastern Restaurant,Discount Store,Coffee Shop,Pizza Place,Chinese Restaurant,Burger Joint,Bus Line,Café
4,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724,2.0,Sandwich Place,Mobile Phone Shop,Bus Line,Pizza Place,Wings Joint,Coffee Shop,Garden Center,Fried Chicken Joint,Fast Food Restaurant,Drugstore


In [22]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Etobicoke_merged['Latitude'], Etobicoke_merged['Longitude'], Etobicoke_merged['Neighborhood'], Etobicoke_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

TypeError: list indices must be integers or slices, not float